In [ ]:
!pip install -r https://raw.githubusercontent.com/EluciDATALab/elucidatalab.starterkits/main/notebooks/SK_3_1_Advanced_Visualisation/requirements.txt
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
import starterkits.starterkit_3_1.support as sp
import starterkits.starterkit_3_1.visualizations as vis

from pathlib import Path
DATA_PATH = Path('../../data/')

%matplotlib inline

# Starter Kit 3.1: Advanced visualisation

## Description

Data visualisation is an important activity in several phases of a data science project. Visualisations allow you to understand the characteristics of a dataset, to discover interesting insights, to validate analysis results, to communicate these results to non-experts via intuitive dashboards, etc.

Popular visualisations often used by data scientists are boxplots, distribution plots, bar and line charts, etc. These are basic out-of-the-box visualisations that are generically applicable, but that are often hard to interpret for non-experts and do not always reveal interesting insights. 

More advanced visualisations exploit the human eye's extraordinary visual pattern recognition abilities. A clever visualisation of data can already reveal interesting patterns and insights, even before any complex algorithm is applied, and can help in formulating hypotheses to be validated further or to identify features that can be useful for the modelling algorithms.

## Business goal

The goal of this Starter Kit is to illustrate more **advanced visualisation techniques**. You will learn how these can be used creatively to uncover more elaborate insights in your data. These insights can then be verified with a domain expert and can be used in subsequent analysis steps, such as feature engineering and data-driven modelling.

## Application context

Advanced visualisations are useful to:
- discover important data properties, e.g. quality-related aspects such as missing values, noise, ...
- reveal hidden patterns, e.g. seasonal behaviour and correlations
- emphasize trends, e.g. degradation of the performance of a machine, increasing popularity of a product or service, ...
- identify interesting cases for further investigation, e.g. outliers, deviations and anomalies
- highlight occurrences of particular events, e.g. machine failures
- verify hypotheses and assumptions, e.g. assumed vs. observed behaviour
- ...

## Starter Kit outline

In this Starter Kit, we will use advanced visualisations such as timeline plots, heatmaps, calendar maps, area plots and scatter plots to visually explore a dataset and gradually build up more knowledge about it. We use a publicly available dataset that consists of bike counter data, to be found [here](https://data.seattle.gov/Transportation/Fremont-Bridge-Hourly-Bicycle-Counts-by-Month-Octo/65db-xm6k), which contains hourly information on the number of bikes that cross 6 spots in Seattle (in two directions). By visualising this data, we will be able to:
- explain certain characteristics of the data, such as some nodes having more crossings than others
- identify global trends, such as an increase in traffic over the years, and seasonal trends, such as fluctuating popularity within a year
- recognize structural patterns, such as distinct weekday and weekend traffic patterns
- detect outliers, such as weekend traffic patterns that occur on weekdays
- ...

## Basic data understanding

The table below shows an excerpt of the data, with the following attributes:
- `Node`: the specific location where a bike counter is located
- `Timestamp`: the timestamp of a measurement, consisting of the date and the time; there is one measurement per hour
- `Date`: the date extracted from the timestamp
- `Time`: the time extracted from the timestamp
- `Hour`: the hour extracted from the timestamp
- `DayOfWeek`: the day of the week (between 1, i.e. Monday, and 7, i.e. Sunday) extracted from the timestamp
- `Direction 0` and `Direction 1`: the number of bike crossings in either one of the two directions at a particular timestamp and a particular node
- `Total`: the total number of bikes crossing at a particular location at a certain timestamp; this is the sum of `Direction 0` and `Direction 1`

In [ ]:
# retrieving and preparing the dataset
data = sp.get_data(DATA_PATH)

In [ ]:
data.head(10)

The summary statistics below show that:
- for each node, we have at least 3 years and a half of data
- the data spans almost 6 years for the Fremont Bridge counter
- the counter on 2nd Avenue is the youngest one with measurements starting in 2015 (despite this fact, it has the third highest total crossings)
- Spokane Street is obviously the 2nd most popular spot after Fremont Bridge, as more than 1 million bikes have passed 
- other nodes are less popular but still got a significant number of bike passings

In [ ]:
data.groupby('Node').agg({'Date': ['min', 'max'], 'Total': 'sum'})

## Geographical visualisation

Let's start by checking where the bike counters are located within the city. We can do that by simply plotting them on a map, which allows us to verify whether they are close to important areas, e.g. tourist or business areas, points-of-interest, etc.

You can click on the red circles to see which node is located where. Considering both Freemont Bridge and Spokane Street are roads that cross a river, it is perhaps not that suprising they have the most crossings: everyone wanting to cross the river needs to go through those nodes, whereas people can choose many different roads in other areas of the city. In addition, NW 58th Street, 39th Av. and 26th Av. are located furthest from the city center, which might explain why they see much less traffic.


In [ ]:
vis.plot_map_nodes(data)

## Timeline visualisation

Since we are dealing with time series data, we can also use a timeline visualisation and plot the average number of crossings per hour and node over time. Such visualisations are useful to:
- observe abrupt changes in the data
- identify time-sensitive trends, such as seasonal variation, increasing or decreasing trends, etc.
- recognize periods with missing data
- ...

The visualisation below just plots the total number of crossings per hour for each node.

In [ ]:
vis.plot_activity_nodes(data);

However, plotting the available data per hour results in a visualisation containing an excessive amount of information (note that data of some nodes hide data of other nodes, a phenomenon called *occlusion*), making it difficult to identify interesting observations in the data such as possible seasonal patterns. In order to resolve this issue, the reader can optionally resample the data to a lower frequency by changing the resampling rate in the corresponding dropdown box.

The seasonal pattern is very clear when using monthly, and to a lesser extent weekly, resampling, while it is not (as) clear when using daily and yearly resampling. For the former, the day-to-day variation makes it harder to see the average trend, while for the latter the seasonal patterns occur over a shorter time period than the time window we use for resampling.

Using a monthly resampling we can clearly see now that:
- Fremont Bridge is the most popular node
- Fremont Bridge is the only node with data in 2012 and 2013. All other nodes start as of 2014, except 2nd Ave which starts in 2015
- A seasonal pattern is clearly visible for Fremont Bridge, Spokane St, and 2nd Ave, where crossings seem to increase in summer and decrease in winter

Such a seasonal pattern might also be present for other, less popular nodes. Large differences in the number of crossings per hour, make it hard to identify patterns for these nodes. We can make the data comparable across nodes by normalising it, that is by changing its numerical values in such a way that a common scale is used without distorting differences in ranges and without losing information. To achieve that, the reader can choose from the corresponding dropdown box to use the *Standard scaling* (also called *Z-score normalisation* or *standardisation*), in which numerical variables are rescaled to have the properties of a standard normal distribution with zero mean and unit variance. The reader might also choose to apply a *Min-max normalisation*, in which the data are rescaled so that the minimum and maximum value correspond to 0 and 1, respectively. Which kind of normalization is best to use in which situation lies outside the scope of this Starter Kit.

Thanks to normalization it becomes clear that the node on 39th Ave also features a similar seasonal pattern. However, 26th Ave and NW 58th St do not follow the same pattern as the other nodes, but do show a seasonal pattern in 2014 which disappears later.

## Heatmap visualisation

An alternative to timeline plots are heatmaps. A heatmap is a visual representation of data that uses colours to indicate varying intensities of values. It is useful to:
- visualise large amounts of data
- reveal spatial/temporal patterns that are much harder to spot in numerical data

Many different types of heatmaps exist:
- matrix heatmaps
- calendar heatmaps
- geographical heatmaps
- circular heatmaps
- ...

### Calendar heatmap

First, we will use a calendar heatmap to visualise time series data on a calendar view. The visualisation below consists of a series of matrix heatmaps (i.e. heatmaps where data is presented in a matrix-like form), in which:
- each year is plotted on a different heatmap row, starting from 2012 at the top and finishing with 2018 at the bottom
- columns in each individual heatmap row correspond to individual weeks within a year
- columns are in turn vertically divided in 7 rows, each corresponding to a different day in a week
- each square corresponds to a single day
- the colour intensity of each square represents the total number of crossings for that day, which in this case are aggregated per day across all nodes

To show the importance of the chosen colour map, we invite the reader to change it and decide for her/himself which one works best in this case.

In [ ]:
vis.plot_calendar_heatmap(data)

This plot provides several interesting insights:
- as the timeline visualisations already revealed, it confirms that bike traffic seems to follow a seasonal pattern, as the winter period (November to May) has different colours each year than the summer period (May to November)
- the last two weeks of the year are always less crowded, which is probably due to the holiday season
- regardless of the specific year and season, weekdays always see more cyclists than weekend days: the last two rows in each matrix heatmap are always lighter than the top rows

We can also see that not all colour maps work equally well. For example, using a circular colormap like *hsv* does not intuitively indicate days with low and high numbers of crossings. *RdBu* does do this better, but it does have an other issue. Diverging colormaps naturally imply a zero-point: blue days have higher values than the zero-point, red days have lower values. Such a zero point is not present in the data, making it not the optimal choice for this case. Sequential colour maps that continuously change from one colour to an other like *YlGn* work much better for this kind of data.

Keep in mind though, that we all might associate different colours with different values. For this, let’s have a look at *RdGn*, a continuously changing colormap from red to green. We usually associate red with bad, dangerous or under-performing datapoints, while green implies the opposite. This is not the case here though. On top of that, this type of colormap might also give problems to people with red–green colour blindness. 

### Matrix heatmap for a single node

A calendar heatmap provides insights into daily patterns and evolution, but we can drill down further and look at whether we can observe some hourly patterns. 

Let's first investigate a single node (i.e. Fremont Bridge) and use a matrix heatmap in which:
- each day of the week is represented as a row
- each hour of the day is represented as a column
- colour intensity indicates the amount of traffic for that day and hour

Note that such a heatmap requires summing the total traffic for each day per hour.

In [ ]:
vis.draw_heatmap_crossings(data, node='Fremont_Bridge')

This plot reveals the following insights:
- it confirms the weekday / weekend day patterns, as the last two rows (representing Saturdays and Sundays) are quite different from the top 5 rows
- there is a clear bimodal pattern for weekdays, with heavier traffic in the morning and evening, not present for weekend days
- traffic in weekdays starts earlier than traffic in weekend days
- traffic in weekdays has some slightly visible darkening in the afternoon
- there is very little traffic at night, which can be expected

The aforementioned bimodal pattern could correspond to a _commute pattern_ corresponding to functional traffic, i.e. people going to work in the morning and returning from work in the afternoon. This is in contrast to the weekend pattern, corresponding to recreational traffic, i.e. people biking for pleasure.

The plot above could also be drawn using a circular heatmap. Such an alternative representation is useful when the values in one of the data dimensions exhibit a circular pattern, like the days of the week (i.e. Sundays, the last day of the week, are followed by Mondays, the first day of the week). The circular heatmap below provides such an alternative representation, in which:
- each day of the week is represented as a concentric circle in the heatmap
- each hour of the day is represented as section on that circle, as on a clock

In [ ]:
vis.draw_circular_heatmap_crossings(data, node='Fremont_Bridge')

### Small multiples

In order to check whether we can see this commute pattern for all nodes, we can use the _small multiples_ technique. A small multiple is a series of graphs of the same type, using the same scale, presented together at a small size with minimal details. The graphs in this view show different partitions of the same dataset, which makes it very easy to compare these different partitions.

Below, we plot a small multiples for the matrix heatmap above, using the different nodes to partition the data. Keep in mind that you should not compare the brightness of the colours between different graphs, as we did not normalise the data (i.e. bringing it to the same scale).


In [ ]:
vis.small_multiples_crossings(data)

We can distinguish the commute pattern that we discovered previously, although with different gradations, in all nodes except for the node on 58th Street. On that node, the graph depicts a different pattern, with a significantly higher number of bikes in the weekend around noon in comparison to weekdays, maybe due to local habits.

## Area plot visualisation

We can investigate the commute pattern even further by taking the direction of the cyclists into consideration. For this, we will use a variant of an area plot called a streamgraph, in which data is displayed around a central axis. In the present case, we will display the average number of crossings per hour at a node around the X axis, in order to discriminate traffic between the two directions. We will plot one direction above the X axis and the other direction below it. The X axis itself will represent the time expressed as hours in a day.

The plot below presents streamgraph plots using the small multiples technique for the different days of the week. The reader can inspect the different nodes, and select the month for which to show the data.

In [ ]:
vis.plot_streamgraph_small_multiples(data)

This plot clearly confirms that the pattern we observed is a real commute pattern for all nodes except NW 58th Street during the week: more cyclists are going through one of the two directions in the morning hours, whereas in the afternoon more cyclists are returning through the other direction. This pattern is absent for weekend days.

NW 58th St does not have a morning and evening peak or a different pattern for week days versus the weekend. It looks more like the weekend days of other nodes for every day of the week. Note that this street is far from the city center, which may explain the absence of commuters.

## Outlier detection

The visualisations presented above already allowed us to identify outliers, e.g. 58th Street seems to behave differently than the other nodes in the network. In this section, we will perform a more detailed analysis of outliers in order to identify them more precisely. 

The table below presents another way of looking at the data: for each node we have 24 observations per day, corresponding to the hourly total number of bike passings through that node.

In [ ]:
pivoted_data_per_node = data.pivot_table(values='Total', index=['Date', 'Node'], columns='Hour').fillna(0)
pivoted_data_per_node.tail(10)

As humans, we cannot easily interpret such a table. However, it is also difficult to visualise such data because it is multi-dimensional: it has 24 values for maximum 6 nodes for thousands of days.

However, we can still visualise that data by first performing a *dimensionality reduction*, using the Principal Components Analysis (PCA) technique. The idea behind PCA is to find a reduced set of axes that summarize the data. More concretely, PCA is a statistical procedure that transforms the multi-dimensional data provided as input into a reduced set of orthogonal (i.e. uncorrelated) dimensions called principal components. 

Using PCA, we can reduce the dimensionality of the data in the above table to two dimensions and plot the result using a scatterplot, as shown below.

In [ ]:
vis.plot_results_pca(pivoted_data_per_node)

Scatterplots typically visualise two variables of a dataset along the X and Y axis, respectively. (Although, as we will see below, a third variable can also be visualised using colour, for example.) Scatterplots are useful to:
- identify relationships between two variables, such as correlation
- identify separate groups in the data, useful for subsequent clustering
- ... 

The scatterplot shown above has the shape of an "L" and seems thus to indicate that there are roughly two groups in the data, corresponding to the two line segments forming the shape of that letter. Through the next figure, we invite the reader to do the analysis for each node individually, and investigate whether these two groups are present for all nodes. From the above analyses, we might expect different results for NW 58th Street and 26th Ave compared to the other nodes.

In [ ]:
vis.plot_pca_per_day(data);

Furthermore, we also offer the possibility to change the colormap indicating the day of the week. We invite the reader to see which option(s) work best for visualizing categorical data like the day of the week.

It should again be clear that some colormaps work better than others. The sequential and divergent colormaps (*YlGn* and *RdBu*) only have one or two colour variations. These colormaps work better for continuous data, such as the heatmaps we had earlier. For categorical data as in this figure, qualitative (also called categorical) colormaps, like *colorblind*,  work much better. As the name suggests, this colormap has the added advantage that it is very readable for people with varying forms of colour blindness.

All in all, we can observe that:
- the two groups are clearly present for Fremont Bridge, Spokane Street, 2nd Avenue, and 39th Avenue, as they show different patterns for weekdays and weekend days: Saturdays and Sundays are located together on the left hand side of the plot, separated from the dots corresponding to the other days
- dots corresponding to Fridays seem to be closer to the weekend pattern
- for NW 58th Street and 26th Avenue, the two groups are not clearly separated, but Saturdays and Sundays typically correspond to points higher in the figure
- a couple of weekdays seem to behave like weekend days: we can observe some of these dots in the upper left part of the plot for all nodes

We suspect these latter dots to be outliers. We can automatically identify them by first applying a clustering algorithm and assigning each data point to one of the two following categories:
- the one formed by the point cloud on the line closer to the Y axis 
- the one formed by the point cloud closer to the X axis 

Then, we can check that the points in each of these two categories correspond to weekend days or weekdays, respectively. Those which do not follow this pattern can be considered outliers.

To cluster (and hence separate) these points we use a Gaussian Mixture Model (GMM), a clustering technique particularly suited to oval-like shapes like those corresponding to the two categories we would like to separate. As training data, we only consider a single node at a time, but in general the model can be trained on the data for all the nodes that exhibit the commute pattern. We only do this for those nodes where the two distinct groups could be identified, so not for NW 58th Street and 26th Avenue. As can be seen from the plot below, the model is quite good in separating the two groups.

In [ ]:
vis.plot_pca_clusters(data)

We can now easily identify the outliers, i.e. those days that are actually weekdays but that are assigned to the weekend day cluster.

The heatmap plot below shows the date corresponding to those outliers. By further checking the US federal holiday calendar, we can see whether the resulting dates corresponded to a holiday, in which case the name of the holiday is indicated on the Y axis.

In [ ]:
vis.plot_heatmap_outliers(data, node='Fremont_Bridge')

We can see that all outliers can be explained: either they are weekdays that are actually holidays or weekdays that are other *special days*, e.g. the Friday after Thanksgiving, the days before and after Christmas, and New Year's Eve.

Note that not all holidays are identified as outliers, e.g. Thanksgiving should occur every fourth Thursday of November, but we did not identify it as outlier in 2016, 2017 and 2018. This might be due to an inaccuracy of the model, which would require a more elaborate preprocessing of the data in order to solve it. This is outside the scope of this Starter Kit.

## Conclusion

In this Starter Kit we have demonstrated how creative visualisations can already reveal interesting patterns and more elaborate insights in your data, even before any complex algorithm is applied. In particular, we have used bike counter data to illustrate how timeline plots, different types of heatmaps, streamgraph plots and scatterplots can be used to: 

- explain certain characteristics of the data, such as some nodes having more crossings than others
- identify global trends, such as an increase in traffic over the years, and seasonal trends, such as fluctuating popularity within a year
- recognize structural patterns, such as distinct weekday and weekend traffic patterns
- detect outliers, such as weekend traffic patterns that occur on weekdays
- ...

These insights can help in formulating hypotheses to be validated further or can be used in subsequent analysis steps, such as feature engineering and data-driven modelling.

## References

The visualisations presented here were originally applied to a private dataset, but the public dataset and some of the exploratory analyses presented in this Starter Kit were inspired by the analysis presented [here](https://jakevdp.github.io/blog/2015/07/23/learning-seattles-work-habits-from-bicycle-counts/).

## Additional information

Copyright © 2022 Sirris

This Starter Kit was developed in the context of the EluciDATA project (http://www.elucidata.be). For more information, please contact info@elucidata.be.

 
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Notebook"), to deal in the Notebook without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Notebook, and to permit persons to whom the Notebook is provided to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies of the Notebook and/or copies of substantial portions of the Notebook.

THE NOTEBOOK IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NON-INFRINGEMENT. IN NO EVENT SHALL SIRRIS, THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, DIRECT OR INDIRECT, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE NOTEBOOK OR THE USE OR OTHER DEALINGS IN THE NOTEBOOK.